In [ ]:
"""
testing.ipynb

File for performing testing to implement lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import functools
from importlib import reload
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow import keras
from keras.callbacks import Callback
from keras import backend as K
from keras import Sequential
from keras.layers import Dense, Input
from keras.losses import CategoricalCrossentropy

from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow_model_optimization.sparsity.keras import ConstantSparsity, PolynomialDecay, prune_low_magnitude

from src.harness.constants import Constants as C
from src.harness.dataset import download_data, load_and_process_mnist
from src.harness.model import save_model, load_model
from src.harness.pruning import create_pruning_callbacks, create_pruning_parameters
from src.harness.training import train, TrainingRound
from src.lottery_ticket.foundations import paths

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Parameters

In [ ]:
X_train, Y_train, X_test, Y_test = load_and_process_mnist()

num_epochs: int = 10
batch_size: int = 60
input_shape: tuple = X_train[0].shape
num_classes: int = 10

num_train_samples: int = X_train.shape[0]

end_step: int = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * num_epochs

pruning_parameters: dict = create_pruning_parameters(0.01, 0, end_step, 100)

## Building

In [ ]:
def set_seed(random_seed: int):
    # Set seeds for reproducability
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

# Create a model with the same architecture using all Keras components to check its accuracy with the same parameters
def create_lenet_300_100(input_shape: tuple[int, ...], num_classes: int, optimizer = C.OPTIMIZER) -> keras.Model:
    """
    Function for creating LeNet-300-100 model.

    :param input_shape: Expected input shape for images.
    :param num_classes: Number of potential classes to predict.
    :param optimizer:   Optimizer to use for training.

    :returns: Compiled LeNet-300-100 architecture.
    """
    model = Sequential([
        Input(input_shape),
        Dense(300, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(100, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(num_classes, activation='softmax', kernel_initializer=tf.initializers.GlorotUniform()),
    ], name="LeNet-300-100")

    model.compile(
        loss=keras.losses.CategoricalCrossentropy(), 
        optimizer=optimizer(), 
        metrics=['accuracy'])

    return model

def global_pruning_nn(input_shape: tuple[int, ...], num_classes: int, pruning_params: dict) -> keras.Model:
    """
    Function to define the architecture of a neural network model
    following 300 100 architecture for MNIST dataset and using
    provided parameter which are used to prune the model.
    
    :param
    """
    
    model = sparsity.prune_low_magnitude(Sequential([
        Input(input_shape),
        Dense(300, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(100, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(num_classes, activation='softmax', kernel_initializer=tf.initializers.GlorotUniform())
    ], name="Pruned_LeNet-300-100"), **pruning_parameters)

    model.compile(
        loss=keras.losses.CategoricalCrossentropy(), 
        optimizer=C.OPTIMIZER(), 
        metrics=['accuracy'])
    
    return model

set_seed(0)
model = global_pruning_nn(input_shape, num_classes, pruning_parameters)
model.summary()

## Pruning

In [ ]:
from src.harness.pruning import get_layer_weight_counts, get_pruning_percents

def count_params(model: keras.Model) -> tuple[int, int]:
    """
    Helper function to count the total number of parameters and number of nonzero parameters.
    """
    weights = model.get_weights()
    total_weights = sum(tf.size(w).numpy() for w in weights)  # Calculate total weights
    nonzero_weights = sum(tf.math.count_nonzero(w).numpy() for w in weights)  # Calculate non-zero weights
    return total_weights, nonzero_weights

def initialize_mask_model(model: keras.Model):
    """
    Function which performs layerwise initialization of a keras model's weights to all 1s
    as an initialization step for masking.

    :param model: Keras model being set to all 1s.
    """
    for weights in model.trainable_weights:
        weights.assign(
            tf.ones_like(
                input=weights,
                dtype=tf.float32,
            )
        )

# Figure out the pruning percents for each round depending on target sparsity and number of pruning rounds
target_sparsity: float = 0.01
layer_weight_counts: list = get_layer_weight_counts(model)
pruning_percents: list[np.array] = get_pruning_percents(layer_weight_counts, .2 ,target_sparsity)


## Training

In [ ]:
from src.harness.experiment import ExperimentData
from src.harness.training import test_step, train, train_one_step, training_loop, TrainingRound

# Save and load model
mask_model = global_pruning_nn(input_shape, num_classes, pruning_parameters)
model: keras.Model = sparsity.strip_pruning(model)

print(f'Nonzero parameters before training:')
print(count_params(model)[1])

experiment_data: ExperimentData = ExperimentData()


In [ ]:
# Test train one step
# Define your loss function outside of the function
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Define your optimizer outside of the function
optimizer = C.OPTIMIZER()


In [ ]:

# Call the function
train_one_step(model, mask_model, X_train, Y_train, loss_fn, optimizer)
model.trainable_variables

In [ ]:
print(f'Nonzero parameters after training:')
print(count_params(model)[1])